In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
cd drive


/content/drive


In [6]:
cd My Drive

/content/drive/My Drive


In [7]:
import keras
import os
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#https://github.com/titu1994/keras-neural-alu/blob/master/nac.py
from keras.engine import Layer
from keras.engine import InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K

from keras.utils.generic_utils import get_custom_objects


class NAC(Layer):
    def __init__(self, units,
                 kernel_W_initializer='glorot_uniform',
                 kernel_M_initializer='glorot_uniform',
                 kernel_W_regularizer=None,
                 kernel_M_regularizer=None,
                 kernel_W_constraint=None,
                 kernel_M_constraint=None,
                 **kwargs):
        """
        Neural Accumulator.
        # Arguments:
            units: Output dimension.
            kernel_W_initializer: Initializer for `W` weights.
            kernel_M_initializer: Initializer for `M` weights.
            kernel_W_regularizer: Regularizer for `W` weights.
            kernel_M_regularizer: Regularizer for `M` weights.
            kernel_W_constraint: Constraints on `W` weights.
            kernel_M_constraint: Constraints on `M` weights.
            epsilon: Small factor to prevent log 0.
        # Reference:
        - [Neural Arithmetic Logic Units](https://arxiv.org/abs/1808.00508)
        """
        super(NAC, self).__init__()
        self.units = units

        self.kernel_W_initializer = initializers.get(kernel_W_initializer)
        self.kernel_M_initializer = initializers.get(kernel_M_initializer)
        self.kernel_W_regularizer = regularizers.get(kernel_W_regularizer)
        self.kernel_M_regularizer = regularizers.get(kernel_M_regularizer)
        self.kernel_W_constraint = constraints.get(kernel_W_constraint)
        self.kernel_M_constraint = constraints.get(kernel_M_constraint)

        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.W_hat = self.add_weight(shape=(input_dim, self.units),
                                     name='W_hat',
                                     initializer=self.kernel_W_initializer,
                                     regularizer=self.kernel_W_regularizer,
                                     constraint=self.kernel_W_constraint)

        self.M_hat = self.add_weight(shape=(input_dim, self.units),
                                     name='M_hat',
                                     initializer=self.kernel_M_initializer,
                                     regularizer=self.kernel_M_regularizer,
                                     constraint=self.kernel_M_constraint)

        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs, **kwargs):
        W = K.tanh(self.W_hat) * K.sigmoid(self.M_hat)
        a = K.dot(inputs, W)
        return a

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'kernel_W_initializer': initializers.serialize(self.kernel_W_initializer),
            'kernel_M_initializer': initializers.serialize(self.kernel_M_initializer),
            'kernel_W_regularizer': regularizers.serialize(self.kernel_W_regularizer),
            'kernel_M_regularizer': regularizers.serialize(self.kernel_M_regularizer),
            'kernel_W_constraint': constraints.serialize(self.kernel_W_constraint),
            'kernel_M_constraint': constraints.serialize(self.kernel_M_constraint),
        }

        base_config = super(NAC, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


get_custom_objects().update({'NAC': NAC})


In [0]:
import numpy as np
import os
import cv2
import keras
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Flatten, Dropout, Input, Bidirectional,BatchNormalization, Reshape
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, Model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed	
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint	
#from nalu import NALU
#from nac import NAC
max_seq = 44
num = np.zeros(6)
path_uiuc = '/content/drive/My Drive/Thesis Codes/'
x = list()
y = list()
#reading color image - changin to gray and subtracting mean
data_fold = ['44D1', '44D2', '44Col']
for fold in data_fold:
    path_44 = path_uiuc + fold + '/'
    print(path_44)
    vidlist = os.listdir(path_44)
    it = len(vidlist)
    for i in range(it):
        s1 = vidlist[i]
        _, typ = s1.split('_')
        #NO REACT
        if typ == '3':
            y.append(3)
        
        elif typ == '0':
           y.append(0)
           
        elif typ == '1':
           y.append(1)
           
        elif typ == '2':
           y.append(2)
           
        elif typ == '4':   
           y.append(4)
           
        elif typ == '5':
           y.append(5)
        else:
            print('Error!' + str(typ))
            
        num[int(typ)] +=1
        
        os.chdir(path_44 + s1)   
        img_list = os.listdir()
        it0 = len(img_list)
        temp = list()
        dim = (32,32)
        for j in range(it0):
            img = cv2.imread(img_list[j])
            print(str(i)+'_'+str(j), end =' ')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            mn1 = np.mean(img)
            st1 = np.std(img)
            img = (img.copy() - mn1) / st1
            img = img.copy().flatten()
            temp.append(img)
        temp = np.asarray(temp)     
        x.append(temp)
        
strokes = {'backhand':num[0], 'forehand':num[1], 'lob':num[2], 'react':num[3], 'serve':num[4], 'smash':num[5]}
for i in strokes.keys():
    print("{} - %d".format(i)%(strokes[i]))
    
x = np.asarray(x)
y = np.asarray(y)
#x = x.reshape(x.shape[0], x.shape[2], x.shape[1])
print(x.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42,shuffle =True)
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
print(X_test.shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
#y_train = np_utils.to_categorical(y_train, 6)
y_test = np_utils.to_categorical(y_test, 6)        


seed = 1   
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
ii = 0 

# create model
i = Input(batch_shape=(None, 44, 1024))
o = NAC(1024, use_gating = True)(i)
o = LSTM(44, return_sequences=False, dropout=0.4)(o)
o = Dense(6, activation='sigmoid')(o)    
model = Model(inputs=[i], outputs=[o])
print(model.summary())
start = time.time()
for train, test in kfold.split(X_train, y_train):
    #print('------------ITERATION : ' + str(ii) + '  -------------------') 
	# Compile model
    #sgd = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    #sgd = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	# Fit the model
    Yy = np_utils.to_categorical(y_train, 6)
    model.fit(X_train[train], Yy[train], epochs=100, batch_size=32, verbose=0)
    model.save('/content/drive/My Drive/Thesis Models/nalu_m'+str(ii) +'.h5')
    ii += 1
    #print(model.predict(X[train], batch_size=32))
	# evaluate the model
    scores = model.evaluate(X_train[test], Yy[test], verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
exectime = time.time()-start    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

final_scores = []
for i in range(5):
  mod = load_model('/content/drive/My Drive/Thesis Models/nalu_m'+str(ii) +'.h5')
  eval1 = time.time()
  scores = model.evaluate(X_test, y_test, verbose=1)
  evaltim = time.time()-eval1
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  final_scores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(final_scores), np.std(final_scores)))
print(exectime, evaltim)

/content/drive/My Drive/Thesis Codes/44D1/
0_0 0_1 0_2 0_3 0_4 0_5 0_6 0_7 0_8 